# Automated ML

Done: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

## Dataset

### Overview
I am using the "Jobs and Salaries in Data Science" dataset from kaggle (https://www.kaggle.com/datasets/hummaamqaasim/jobs-in-data?resource=download). It contains job descriptions (e.g., Data Engineer, Data Scientist) and salaries (both in original currency and USD), experience level, type of employment (full time vs part-time), and work setting (in-person/hybrid). For the capstone project I want to predict the salary in USD using a regression model. It's important to delete the "salary" column (in original currency) beforehand, as it is almost the same as the target column and should not be an input of the model.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'my-automl-experiment'

experiment=Experiment(ws, experiment_name)



In [3]:
found = False
key = "DataScientistSalaries"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/sarahsofia93/udacity-MLengineer-capstone/blob/master/jobs_in_data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()



,work_year,salary_in_usd
count,9355.000000,9355.000000
mean,2022.760449,150299.495564
std,0.519470,63177.372024
min,2020.000000,15000.000000
25%,2023.000000,105700.000000
50%,2023.000000,143000.000000
75%,2023.000000,186723.000000
max,2023.000000,450000.000000


In [4]:
amlcompute_cluster_name = "my-fancy-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

(Done): Explain why you chose the automl settings and cofiguration you used below.

I am choosing a regression model since my target variable (salary) is a continuous rather than a categorial variable. I am using the primary metric R2 score which is a common metric for regresisons in AutoML. It also helps me to establish comparability with the hyperdrive model, where I also use the r aquared as metric.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'r2_score',
    "n_cross_validations": 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target,
                             task = "regression",
                                training_data = dataset,
                                label_column_name = "salary_in_usd",
                                path = "./automl-run",
                                enable_early_stopping= True,
                                featurization = "auto",
                                debug_log = "automl_errors.log",
                                **automl_settings
)


In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)


Submitting remote run.
No run_configuration provided, running on my-fancy-cluster with default configuration
Running on remote compute: my-fancy-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
my-automl-experiment,AutoML_dbdd65df-e1ce-4515-a57c-23b8e29675df,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https:/

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_model, best_model_metrics = remote_run.get_output()
best_model.get_properties()
print(best_model)
best_model_metrics = best_model.get_metrics()

for metric_name in best_model_metrics:
    metric = best_model_metrics[metric_name]
    print(metric_name, metric)



Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: my-automl-experiment,
Id: AutoML_dbdd65df-e1ce-4515-a57c-23b8e29675df_36,
Type: azureml.scriptrun,
Status: Completed)
explained_variance 0.3856050556491951
r2_score 0.38514080554422836
normalized_mean_absolute_error 0.08690432082155974
normalized_median_absolute_error 0.0702146838141003
spearman_correlation 0.6222380785769575
root_mean_squared_error 49496.196175007455
root_mean_squared_log_error 0.3409368381207109
normalized_root_mean_squared_log_error 0.10024213819090252
normalized_root_mean_squared_error 0.11378435902300563
median_absolute_error 30543.387459133628
mean_absolute_percentage_error 29.608013754315294
mean_absolute_error 37803.37955737848
predicted_true aml://artifactId/ExperimentRun/dcid.AutoML_dbdd65df-e1ce-4515-a57c-23b8e29675df_36/predicted_true
residuals aml://artifactId/ExperimentRun/dcid.AutoML_dbdd65df-e1ce-4515-a57c-23b8e29675df_36/residuals


In [9]:
#TODO: Save the best model
import joblib
#os.makedirs('./outputs', exist_ok=True)
#joblib.dump(best_model, filename='outputs/best_automl_model.joblib')

myModel = best_model.register_model(model_name='best_automl_model', model_path='outputs/model.pkl')


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
# Download scoring file 
best_model.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_model.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

from azureml.automl.core.shared import constants
best_model.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [11]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core import Model

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_model.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-02 08:51:31+00:00 Creating Container Registry if not exists.
2024-02-02 08:51:32+00:00 Registering the environment.
2024-02-02 08:51:32+00:00 Use the existing image.
2024-02-02 08:51:32+00:00 Submitting deployment to compute.
2024-02-02 08:51:38+00:00 Checking the status of deployment myservice..
2024-02-02 08:53:42+00:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://f39424c3-dd9f-484d-a7ca-987c2aef90e2.westus2.azurecontainer.io/score
http://f39424c3-dd9f-484d-a7ca-987c2aef90e2.westus2.azurecontainer.io/swagger.json


TODO: In the cell below, print the logs of the web service and delete the service

In [16]:
import json


# df is the data as seen by the model during training
df = pd.read_csv('jobs_in_data.csv')
df = df.drop(columns=['salary_currency', 'salary'])


test_df = df.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('salary_in_usd')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)


{"data": [{"work_year": 2023, "job_title": "Machine Learning Engineer", "job_category": "Machine Learning and AI", "employee_residence": "United States", "experience_level": "Senior", "employment_type": "Full-time", "work_setting": "In-person", "company_location": "United States", "company_size": "M"}, {"work_year": 2023, "job_title": "Machine Learning Engineer", "job_category": "Machine Learning and AI", "employee_residence": "United States", "experience_level": "Senior", "employment_type": "Full-time", "work_setting": "Remote", "company_location": "United States", "company_size": "M"}, {"work_year": 2023, "job_title": "Data Analyst", "job_category": "Data Analysis", "employee_residence": "United States", "experience_level": "Senior", "employment_type": "Full-time", "work_setting": "Remote", "company_location": "United States", "company_size": "M"}, {"work_year": 2022, "job_title": "Applied Scientist", "job_category": "Data Science and Research", "employee_residence": "United States",

In [17]:
import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)


In [18]:
# Print results from the inference
print(response.text)
# Print original labels
print(label_df)

print(service.get_logs())



"{\"result\": [198047.17480941652, 197559.10978286812, 122619.30385726779, 187788.51788121482, 133882.38200918754]}"
1967    132600
4401    235400
5522    208450
7589    142000
115     184000
Name: salary_in_usd, dtype: int64
2024-02-02T08:53:20,587368600+00:00 - rsyslog/run 
2024-02-02T08:53:20,596313900+00:00 - gunicorn/run 
2024-02-02T08:53:20,599018400+00:00 | gunicorn/run | 
2024-02-02T08:53:20,605475300+00:00 | gunicorn/run | ###############################################
2024-02-02T08:53:20,607299100+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-02-02T08:53:20,613324600+00:00 | gunicorn/run | ###############################################
2024-02-02T08:53:20,614830600+00:00 | gunicorn/run | 
2024-02-02T08:53:20,623472200+00:00 | gunicorn/run | 
2024-02-02T08:53:20,623721800+00:00 - nginx/run 
2024-02-02T08:53:20,632761300+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-02-02T08:53:20,63424160

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
